In [ ]:
import torch
import numpy as np
from sklearn.preprocessing import StandardScaler
import xlrd
import pandas as pd

#original data used to stadarise replicated data
book = xlrd.open_workbook('dataset for standardisation.xlsx')
sheet = book.sheet_by_name('Sheet1')
data_unreplicated = [[sheet.cell_value(r, c)
         for c in range(1,9)] for r in range(2,74)]
scaler = StandardScaler()
scaler.fit(data_unreplicated)

#replicated data
book = xlrd.open_workbook('prepareddata.xlsx')
sheet = book.sheet_by_name('Sheet1')
replicated_data = [[sheet.cell_value(r, c)
         for c in range(0,8)] for r in range(0,7272)]

#standardise
standardised_data = scaler.transform(replicated_data)
np.random.shuffle(standardised_data)
#split into inputs and targets
array1 = standardised_data[:, 0:5]
array2 = standardised_data[:, 5:8]
x = torch.tensor(array1)
y = torch.tensor(array2)



In [ ]:
#define model
import torch.nn as nn
class Net(nn.Module):
    def __init__(self, D_in, H1, H2, D_out):
 
        super(Net, self).__init__()
        self.linear1 = nn.Linear(D_in, H1) 
        self.linear2 = nn.Linear(H1, H2) 
        self.linear3 = nn.Linear(H2, D_out)
  
    def forward(self, x):
    
        x = torch.sigmoid(self.linear1(x))
        x = torch.sigmoid(self.linear2(x))
        x = self.linear3(x)
        return x



In [ ]:
# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H1, H2, D_out = 100, 5, 15, 9, 3
model = Net(D_in, H1, H2, D_out)
loss_fn = torch.nn.MSELoss(reduction='mean')

In [ ]:
#put into batches
train = torch.utils.data.TensorDataset(x, y)
train_loader = torch.utils.data.DataLoader(train, batch_size=N, shuffle=True)
model.train()
#define remaining hyperparameters
train_epochs=100
learning_rate=1e-3
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

#train
for epoch in range(train_epochs):
    avg_loss=0
    for x_batch, y_batch in train_loader:
        y_pred = model(x_batch.float())
        loss = loss_fn(y_pred, y_batch.float())
        avg_loss += loss/ len(train_loader.dataset)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(epoch,avg_loss)

In [ ]:
#save trained model
torch.save(model.state_dict(), ("C:/Users/Gabriel/Documents/Python/ANN2.pt"))                
                                